In [1]:
import TSD
import altair as alt
import pandas as pd
import numpy as np
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta
import random
from altair.expr import datum, if_


In [2]:
def compute_age(row): #and round to five
    d1 = row["start"]
    d2 = row["dob"]
    diff_in_years = relativedelta(d1, d2)
    return_val = math.ceil(diff_in_years.years / 5) * 5
    if return_val == 0: return_val = 5
    return return_val

def compute_age_line(row):
    d1 = row["start"]
    d2 = row["dob"]
    diff_in_years = relativedelta(d1, d2)
    return diff_in_years.years



In [3]:
merged_df = TSD.df_merged.copy()
merged_df["age_in_years"] = merged_df.apply(lambda row: compute_age(row), axis=1)

#calculate average per clinic historically
merged_df_2 = TSD.df_merged.copy()
merged_df_2["age_in_years"] = merged_df_2.apply(lambda row: compute_age_line(row), axis=1)

clinic_counts = merged_df_2.groupby(['clinic_id','age_in_years']).size().reset_index(name='count')
avg_clinic = clinic_counts.groupby('age_in_years').mean().drop('clinic_id', axis=1).reset_index()
avg_clinic.age_in_years = avg_clinic.age_in_years // 5
grouped_avg = avg_clinic.groupby('age_in_years').sum().reset_index()
grouped_avg.age_in_years = grouped_avg.age_in_years * 5

#find number of clinics
total_clinics = merged_df.clinic_id.nunique()



In [4]:
clinics = {}
totals_by_age_clinic = {}
condition_total = {}

exploded_df = merged_df.explode('Category')


for index, row in exploded_df.iterrows():
    # Create a tuple of (clinic_id, age_in_years) to identify clinics uniquely
    
    #calculate totals_by_age_clinic
    the_key = (row.clinic_id,row.age_in_years)
    if the_key in condition_total:
        condition_total[the_key] += 1
    else:
        condition_total[the_key] = 1  
        
        
for index, row in merged_df.iterrows():
    # Create a tuple of (clinic_id, age_in_years) to identify clinics uniquely
   
    clinic_key = (row.clinic_id, row.age_in_years, row.gender)
        
    for cat in row.Category:
        new_key = (clinic_key[0],clinic_key[1],clinic_key[2],cat)
        if new_key in clinics:
            clinics[new_key] += 1
        else:
            clinics[new_key] = 1
    
    #calculate totals_by_age_clinic
    second_key = (clinic_key[0],clinic_key[1])
    if second_key in totals_by_age_clinic:
        totals_by_age_clinic[second_key] += 1
    else:
        totals_by_age_clinic[second_key] = 1  


dummy_df = pd.DataFrame()
rows = []

##percent of total for age/clinic


for key, count in clinics.items():
    #debug = False
    #if(key[0]==32 and key[1]==15):
    #    print("***")
    #    debug = True
    new_row = {}
    new_row['clinic_id'] = key[0]
    new_row['age'] = key[1]
    new_row['Category'] = key[3]
    new_row['Count'] = count
    rows.append(new_row)
    percent = count/totals_by_age_clinic[(key[0],key[1])]
    #check for other gender version and add together
    gender = key[2]
    key_opp = (key[0],key[1],'m' if gender == 'f' else 'f',key[3])
    key_opp_count = 0
    if key_opp in clinics:
        key_opp_count = clinics[key_opp]
        percent = percent + key_opp_count/totals_by_age_clinic[(key_opp[0],key_opp[1])]
    new_row['Percent*'] = str(round(100*percent,2))+"%"
    
    #to calculate raw - check if there are both male and female
    #if there are male = 0 for raw as its used for the total display
    #if not just include the one
    new_row['raw'] = 0
    if key_opp_count > 0 and count > 0:
        #both have check if female
        if gender == 'f':
            new_row['raw'] = count/condition_total[(key[0],key[1])] + key_opp_count/condition_total[(key[0],key[1])]
    else: #don't have both
        new_row['raw'] = count/condition_total[(key[0],key[1])]
            
    
    if gender == 'f':
        gender_text = str(key_opp_count)+"/"+str(count)
    else:
        gender_text = str(count)+"/"+str(key_opp_count)
    new_row['gender'] = gender_text
    #if debug: print (new_row)
dummy_df = pd.DataFrame(rows)
#clinic_id, age, category, percent of total for age/clinic, Gender M/F

#duplicate rows not sure how else do do this
rows = []

# Define a function for the row-wise operation
def calculate_display(row):
    return_val = row['raw'] * totals_by_age_clinic.get((row.clinic_id, row.age), 0)
    return return_val

# Apply this function to each row
dummy_df['display'] = dummy_df.apply(calculate_display, axis=1)



final_df = dummy_df.copy()
final_df = final_df.rename(columns={'gender': 'Gender M/F*'})
final_df['age'] = final_df['age'] - 1



In [5]:
#Calculate Summaries

#setup dates in final_df for Dropdown
date_format = "%d %B %Y"



#Other
summary_raw = merged_df.copy()

summary_raw['clinic_dates'] = summary_raw['clinic_id']
clinic_df = TSD.df_clinic.copy()
replace_dict = clinic_df.set_index('id')['start'].to_dict()
summary_raw['clinic_dates'] = summary_raw['clinic_dates'].replace(replace_dict)
summary_raw['clinic_dates'] = pd.to_datetime(summary_raw['clinic_dates']).dt.strftime(date_format)

summary_raw = summary_raw.drop(['dob','city','colonia','state','timein','timeout','patient_id','location','start','end','Category'], axis=1)
def gender_to_num(row) :
    return 0 if row['gender'] == 'm' else 1
summary_raw['gender'] = summary_raw.apply(gender_to_num, axis=1)
summary_raw['count'] = 1

# Grouping by 'Category' and applying different aggregation functions
summary_info = summary_raw.groupby('clinic_dates').agg({
    'gender': 'mean',  
    'clinic_id': 'first',  
    'age_in_years': 'mean',
    'count': 'sum'
})

summary_info = summary_info.reset_index()

#Xrays

xrays = TSD.df_xray.copy()
xrays['count'] = 1
xrays = xrays.drop(['id','imagetype','path','timestamp','patient_id','station_id'],axis=1)
xrays = xrays.groupby('clinic_id').sum().reset_index()

xrays['clinic_dates'] = xrays['clinic_id']
xrays['clinic_dates'] = xrays['clinic_dates'].replace(replace_dict)
xrays['clinic_dates'] = pd.to_datetime(xrays['clinic_dates']).dt.strftime(date_format)




#this is the sumary for each clinic
def compute_summary_string (row):
    percent_m = str(round(100*(1-row['gender']),1))
    percent_f = str(round(100*(row['gender']),1))
    xray_amount = '--'

    for xray_row in xrays.itertuples():
       if xray_row.clinic_dates == row.clinic_dates:
            xray_amount = str(xray_row.count)
    age_r = str(round(row['age_in_years'],2))
    summary_details = 'Patients: '+str(row['count'])+'   Average Age: '+age_r+ '   Male: '+percent_m+'%'+ \
    '   Female: '+percent_f+'%' + '   X-rays: '+xray_amount
    return summary_details

def compute_title_string (row):
  

    summary_title = "Clinic Statistics for "+row['clinic_dates']
    return summary_title

#Summary Statistics
summary_info['summary'] = summary_info.apply(compute_summary_string, axis = 1)
summary_info['title'] = summary_info.apply(compute_title_string, axis = 1)

summary_info
    


,clinic_dates,gender,clinic_id,age_in_years,count,summary,title
0,01 August 2021,0.607143,16,16.726190,84,Patients: 84 Average Age: 16.73 Male: 39.3...,Clinic Statistics for 01 August 2021
1,01 May 2021,0.625000,15,17.812500,16,Patients: 16 Average Age: 17.81 Male: 37.5...,Clinic Statistics for 01 May 2021
2,03 February 2023,0.538462,30,12.948718,156,Patients: 156 Average Age: 12.95 Male: 46....,Clinic Statistics for 03 February 2023
3,03 November 2021,1.000000,17,5.000000,1,Patients: 1 Average Age: 5.0 Male: 0.0% ...,Clinic Statistics for 03 November 2021
4,04 August 2023,0.570248,32,14.958678,121,Patients: 121 Average Age: 14.96 Male: 43....,Clinic Statistics for 04 August 2023
5,04 November 2022,0.537313,25,11.343284,134,Patients: 134 Average Age: 11.34 Male: 46....,Clinic Statistics for 04 November 2022
6,05 August 2022,0.590909,24,16.761364,88,Patients: 88 Average Age: 16.76 Male: 40.9...,Clinic Statistics for 05 August 2022
7,05 January 2023,0.714286,28,7.857143,7,Patients: 7 Average Age: 7.86 Male: 28.6% ...,Clinic Statistics for 05 January 2023
8,05 May 2023,0.601626,31,14.674797,123,Patients: 123 Average Age: 14.67 Male: 39....,Clinic Statistics for 05 May 2023
9,06 May 2022,0.529801,23,15.298013,151,Patients: 151 Average Age: 15.3 Male: 47.0...,Clinic Statistics for 06 May 2022


In [6]:
#counts for largest age to set y axis
yMax = math.ceil(np.max(final_df.groupby(['clinic_id','age']).display.sum()))

#setup dates in final_df for Dropdown
date_format = "%d %B %Y"

final_df['clinic_dates'] = final_df['clinic_id']
df = TSD.df_clinic.copy()
replace_dict = df.set_index('id')['start'].to_dict()
final_df['clinic_dates'] = final_df['clinic_dates'].replace(replace_dict)
final_df['clinic_dates'] = pd.to_datetime(final_df['clinic_dates']).dt.strftime(date_format)

clinic_dates = final_df.clinic_dates.unique()

# Convert date strings to datetime objects
date_objects = np.array([datetime.strptime(date, date_format) for date in clinic_dates])

# Sort datetime objects in descending order
sorted_dates = np.sort(date_objects)[::-1]

# Convert sorted datetime objects back to date strings
clinic_dates = [date.strftime(date_format) for date in sorted_dates]

# Custom colors for each category 
category_colors = {
    'Unlisted': '#F06060',
    'Trouble Hearing': '#F3B562',
    'Cold Cough Fever': '#F2EBBF',
       'Born With Cleft Lip': '#8CBEB2',
    'Born With Cleft Palate': '#5C4B51',
       'Trouble Speaking': '#8C8474',
    'Congenital Heart Defect': '#BEBF95',
    'Trouble Eating' : '#8C2B59',
       'Bleeding Problems': '#57315A',
    'Hepititis': '#333259',
    'Diabetes': '#001143',
    'Anemia': '#3C4263',
    'Athsma': '#28708C',
       'Epilepsy': '#EBC62A'
}



# A dropdown filter
clinic_dropdown = alt.binding_select(options=clinic_dates, name="Clinic Date:")
clinic_select = alt.selection_point(fields=['clinic_dates'], bind=clinic_dropdown, \
                                    value=clinic_dates[0])

#Chart most recent clinic with line showing historical average
line = alt.Chart(grouped_avg).mark_line(color='blue', strokeOpacity=0.35).encode(
    alt.X('age_in_years:Q',title='Age in Years',bin = alt.Bin(maxbins=20)),
    y = 'count:Q',
)



base = alt.Chart(final_df).encode (
       x =    alt.X('age', bin = alt.Bin(maxbins=20))   
)
bars = base.mark_bar().encode(
    y = alt.Y('display',title='Number of Patients',scale=alt.Scale(domain=[0,yMax])),
    color=alt.Color('Category', scale=alt.Scale(domain=list(category_colors.keys()),
                                                range=list(category_colors.values()))),
    tooltip=['Category:N','Percent*:N','Gender M/F*:N']).add_params(
    clinic_select
).transform_filter(
    clinic_select
)

#Add summary
#Summary Chart
summary = alt.Chart(summary_info).mark_text(fontSize=15).encode(
text = 'summary',
).add_params(
    clinic_select
).transform_filter(
    clinic_select
).properties(
        width=700,
        height=50  # Adjust height as needed
    )

title = alt.Chart(summary_info).mark_text(fontSize=20, fontWeight='bold').encode(
text = alt.Text('title')
).add_params(
    clinic_select
).transform_filter(
    clinic_select
).properties(
 width=700,
)

main_layer = alt.layer(bars, line).properties(
    width=700,
    height=500)

chart = alt.vconcat(title, main_layer, summary.properties(
        width=700,
        height=50  # Adjust height as needed
    ))

chart.save('TS-Demographics.html')
chart

alt.VConcatChart(...)

In [7]:

#double check totals
print("Total Data Rows:",len(TSD.df_merged))

#calculate total display
total_display = np.sum(final_df.display)
print("Total Displayed:",total_display)



Total Data Rows: 1026
Total Displayed: 1026.0
